In [1]:
import torch
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

# Loading and saving models

In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os 

processor = WhisperProcessor.from_pretrained("bofenghuang/whisper-large-v2-cv11-german")
model = WhisperForConditionalGeneration.from_pretrained("bofenghuang/whisper-large-v2-cv11-german")
model.config.forced_decoder_ids = None

# Assuming `model` is your trained model and `processor` is your processor
save_directory = "models/whisper-large-v2-cv11-german"

# Create output directory if needed
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the model
model.save_pretrained(save_directory)

# Save the processor/tokenizer
processor.save_pretrained(save_directory)

c:\Users\svena\Documents\GitHub\audioTranscribe\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# With manual setting of language:

In [5]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer, pipeline
import torch
import librosa
import soundfile as sf

feature_extractor = WhisperFeatureExtractor.from_pretrained("models/whisper-large-v2-cv11-german")
tokenizer = WhisperTokenizer.from_pretrained("models/whisper-large-v2-cv11-german", language="german", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained("models/whisper-large-v2-cv11-german")
forced_decoder_ids = tokenizer.get_decoder_prompt_ids(language="de", task="transcribe")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Device:", device)

asr_pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    chunk_length_s=30,
    stride_length_s=10,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


Device: cuda:0


In [8]:
from pydub import AudioSegment

AudioSegment.converter = "C:/ffmpeg/bin/ffmpeg.exe"

# Specify the path to your local audio file
audio_file = "demoFiles/InterviewMuenchen.mp3"

# Load audio file
audio, sr = librosa.load(audio_file, sr=None)

# Slice the first minute
one_minute_audio = audio[:sr*60]  # librosa works in samples, sr*60 gives the number of samples in one minute
# one_minute_audio = audio[:sr*60]  # librosa works in samples, sr*60 gives the number of samples in one minute

# Save to a temporary file
temp_file = "temp.wav"
sf.write(temp_file, one_minute_audio, sr)

print("Audio file saved to", temp_file)

# Use the temporary file for prediction
prediction = asr_pipe(temp_file, batch_size=1, generate_kwargs={"forced_decoder_ids": forced_decoder_ids})

print(prediction['text'])

Audio file saved to temp.wav
Be er Heimat, habe die Ehre! Ich begrüße Sie ganz herzlich. Zehn Uhr ist es die Zeit. Könnte man schon an ein Weißwurstfrühstück denken? Ja, da haben Sie recht. Das wäre jetzt genau das Richtige. Leider können wir Sie übers Radio nicht servieren, aber ein bisschen über die Weißwurst reden, das können wir natürlich schon. Denn wenn Sie in München fragen, wo gehe ich denn hin, wo kriege ich denn richtig gute Weißwurst, würden viele Münchner sagen, ja, da musst zum Wallner in die Gaststätte Großmarkthalle in Sendling im Bauch von München. Und der Wirt, der Ludwig Wallner, der ist heute bei uns zu Gast. Schön, dass Sie sich zeichnommen haben, da freuen wir uns. Grüß Gott miteinander, oder grüß Gott. Vielen Dank für die recht herzliche Einladung. Also wir haben jetzt hier bloß einen Kaffee stehen unter Wasser. und wie wär das jetzt die Uhrzeit? Sie fangen sehr früh an, haben Sie mir erzählt, kurz nach fünf ist es schon in Betrieb, der macht ja auch sieben auf. H